# Graph Neural Networks (GNNs) Basics

In this notebook, we’ll explore the fundamentals of **Graph Neural Networks (GNNs)** : a powerful class of models designed to work directly on graph-structured data. 

Graphs are ubiquitous in the real world: they represent **social networks, molecular structures, citation networks, and recommendation systems**. Unlike images or text, graphs have an **irregular structure**, which makes them challenging for traditional deep learning models.

## 🚀 Learning Objectives
- Understand what graphs are and why GNNs are needed.
- Learn about message passing and graph convolution.
- Build a simple GCN (Graph Convolutional Network) model using PyTorch Geometric.
- Apply GNNs to a sample dataset (Cora citation network).

## 📘 1. What is a Graph?

A **graph** is defined as \( G = (V, E) \):
- **V** → Set of nodes (vertices)
- **E** → Set of edges (connections between nodes)

Each node can have **features**, and edges can represent relationships or similarities.

Example:
```
A --- B
|     |
C --- D
```
Here A, B, C, D are nodes, and the connecting lines are edges.

## ⚙️ 2. Why Use GNNs?

Unlike CNNs that work on grid data (images), GNNs learn **node embeddings** by aggregating information from a node’s **neighbors**.

This process is called **message passing**:

\[
h_v^{(k)} = \sigma \Big( W^{(k)} \cdot AGGREGATE(\{h_u^{(k-1)} : u \in N(v)\}) \Big)
\]

Where:
- \( h_v^{(k)} \) → embedding of node *v* at layer *k*
- \( N(v) \) → neighbors of node *v*
- \( \sigma \) → activation function
- \( W^{(k)} \) → learnable weights

In [ ]:
# Install PyTorch Geometric (if running locally)
# !pip install torch-geometric torch-scatter torch-sparse torch-cluster
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv

print('Libraries imported successfully!')

## 🧩 3. Load the Cora Dataset

The **Cora dataset** is a citation network where:
- Nodes = research papers
- Edges = citations
- Features = bag-of-words representations
- Task = classify papers into 7 categories

In [ ]:
dataset = Planetoid(root='data/Cora', name='Cora')
data = dataset[0]

print('Dataset:', dataset)
print('Number of nodes:', data.num_nodes)
print('Number of edges:', data.num_edges)
print('Number of features:', data.num_features)
print('Number of classes:', dataset.num_classes)

## 🏗️ 4. Building a Simple Graph Convolutional Network (GCN)

A GCN applies convolution-like operations on graphs. Each layer aggregates and transforms neighbor features.

In [ ]:
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
print(model)

## 🎯 5. Training the GCN

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

for epoch in range(200):
    loss = train()
    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}')

## 📊 6. Evaluate the Model

In [ ]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Test Accuracy: {acc:.4f}')

## 🧭 7. Key Takeaways
- GNNs generalize deep learning to graph data.
- Message passing allows nodes to aggregate information from neighbors.
- GCNs are one of the foundational GNN architectures.
- Frameworks like **PyTorch Geometric** simplify GNN implementation.

## 🧠 Further Exploration
- GraphSAGE: inductive node embeddings.
- GAT (Graph Attention Networks): attention-based aggregation.
- Applications: molecular property prediction, traffic forecasting, recommendation systems.